In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.gate.io/trade/BLOCK_USDT'

response = requests.get(url)
# 사이트를 통해 모든 코인 데이터를 받아온다. 
if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    print(soup)

else : 
    print(response.status_code)

In [1]:
# !/usr/bin/env python
# coding: utf-8
import logging
import random
import os
import csv
import numpy as np
import pandas as pd
import datetime 
import sys
import time
import threading
import concurrent.futures
import sqlite3

from decimal import Decimal as D, ROUND_UP, getcontext
# gate api를 통해 코인 리스트를 가져온다. 
from gate_api import ApiClient, Configuration, Loan, MarginApi, Order, RepayRequest, SpotApi, Transfer, WalletApi
from gate_api.exceptions import GateApiException

config = Configuration(key="", secret="")
spot_api = SpotApi(ApiClient(config))


In [2]:
# 코인 리스트 중에서 위의 코인과 비교 진행 
# BULL과 BEAR, 3L, 3S 
# 를 제외한 코인 검색진행 
coin_list = spot_api.list_currency_pairs()
# print(coin_list)
spots = []
# find 함수는 있으면 있는 곳의 위치를 없으면 -1 를 반환 만약 첫째 자리에 있다면 0을 반환 
for c_l in coin_list :
    if c_l.id.find('USDT') != -1 and  c_l.id.find('BULL') == -1 and c_l.id.find('BEAR') == -1 and c_l.id.find('3L') == -1 and c_l.id.find('3S') == -1:
        spots.append(c_l.id)
    

In [3]:
from gateio_market_cur import  *
from gateio_coins import * 

old_coin = marketlist_all
# gate io의 코인 리스트 쭉 가져오기
all_coin = marketlist_all_currs['spot']

In [4]:
new_coinlist = {}
for coin in all_coin :
    if coin['symbol']+'_USDT' in spots : 
        # if '3L' in coin['symbol'] :
        #     print(coin['symbol'])
        new_coinlist[coin['symbol']+'_USDT'] = coin['name_en']

In [ ]:
print(new_coinlist)

In [7]:
# 신규 코인과 현재 저장된 코인에서 공통으로 포함되어 있지 않은 코인은 제거 
new_addcoin = {}
for n_c in new_coinlist.keys() :
    if n_c not in marketlist_all.keys() :
        new_addcoin[n_c] = new_coinlist[n_c]
    
print(new_addcoin)

{'GARI_USDT': 'GARI Network', 'ASTR_USDT': 'Astar Token', 'GHC_USDT': 'Galaxy Heroes Coin', 'LOOKS_USDT': 'LooksRare', 'GLMR_USDT': 'Moonbeam Network', 'POKT_USDT': 'Pocket Network', 'SLC_USDT': 'Solice', 'FRR_USDT': 'Frontrow', 'CRTS_USDT': 'Cratos', 'GFI_USDT': 'Goldfinch', 'SFM_USDT': 'SafeMoon', 'ROCO_USDT': 'Roco Finance', 'NOS_USDT': 'Nosana', 'BEYOND_USDT': 'Beyond Protocol', 'ZIG_USDT': 'Zignaly', 'NCT_USDT': 'PolySwarm', 'VOXEL_USDT': 'Voxies', 'CLH_USDT': 'ClearDAO', 'UFI_USDT': 'PureFi', 'KASTA_USDT': 'Kasta', 'SOLO_USDT': 'Sologenic', 'ALPH_USDT': 'Alephium', 'ZODI_USDT': 'Zodium', 'GCOIN_USDT': 'Galaxy Fight Club', 'DOMI_USDT': 'Domi', 'MBS_USDT': 'MonkeyBall', 'REVU_USDT': 'Revuto', 'ISKY_USDT': 'Infinity Skies', 'SVT_USDT': 'Solvent', 'CHICKS_USDT': 'SolChicks', 'APX_USDT': 'ApolloX', 'FRM_USDT': 'Ferrum Network', 'GENS_USDT': 'Genshiro', 'F2C_USDT': 'Ftribe Fighters Coin', 'APT_USDT': 'Apricot', 'ONIT_USDT': 'ONBUFF', 'ARTEM_USDT': 'ARTEM Coin', 'STSOL_USDT': 'Lido Stak

In [51]:
checkAPI_coin_id(symbol = 'ETC' )

[{'id': 1321,
  'name': 'Ethereum Classic',
  'symbol': 'ETC',
  'slug': 'ethereum-classic',
  'rank': 41,
  'is_active': 1,
  'first_historical_data': '2016-07-24T13:44:30.000Z',
  'last_historical_data': '2021-12-31T15:59:00.000Z',
  'platform': None},
 {'id': 646,
  'name': 'Ethercoin',
  'symbol': 'ETC',
  'slug': 'ethercoin',
  'rank': None,
  'is_active': 0,
  'first_historical_data': '2014-10-03T12:34:22.000Z',
  'last_historical_data': '2015-08-16T13:34:19.000Z',
  'platform': None}]

In [ ]:
# 코인 마켓캡에서 데이터를 가져온다. 
from coinmarketcap import *
# GATEIO의 든 코인의 coinmarketcap id를 가져온다. 
# 우선 저장된 코인 구별 진행 
import time 

id_list = {}
n_c = []
for nc in new_addcoin.keys() :
    if nc not in coin_cap_id.keys() :
        n_c.append(nc)

# print(n_c)
for cp in n_c :
    symbol = cp.replace("_USDT", "")
    print(symbol)
    coin_name = new_addcoin[cp].replace(" ", "").replace(".", "").replace("-","").replace("_","").lower()
    data = checkAPI_coin_id(symbol = symbol )
    if data :
        if len(data) == 1 :
            for _data in data :
                print("해당됨")
                print(coin_name)
                print(_data['slug'].lower().replace("-", "").replace(" ", ""))
                id_list[cp] = _data['id']
                break
        else :
            for _data in data :
                time.sleep(0.2)
                if coin_name in _data['slug'].lower().replace("-", "").replace(" ", "") or coin_name in _data['name'].replace(" ","").lower() \
                    or  _data['name'].replace(" ","").lower() in coin_name or  _data['slug'].replace(" ","").lower() in coin_name :
                    print("해당됨")
                    print(coin_name)
                    print(_data['slug'].lower().replace("-", "").replace(" ", ""))
                    id_list[cp] = _data['id']
                    break
            
print(id_list)

In [30]:
print( len(id_list.keys()) )

737


In [8]:
# 코인 마켓캡에서 데이터를 가져온다. 
from coinmarketcap import *
# GATEIO의 든 코인의 coinmarketcap id를 가져온다. 
# 우선 저장된 코인 구별 진행 
import time 

data = checkAPI_coin_id(symbol = 'near' )

data = checkAPI_info( [ 6535 ] )
data

{'status': {'timestamp': '2022-01-19T15:29:48.343Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 16,
  'credit_count': 1,
  'notice': None},
 'data': {'6535': {'id': 6535,
   'name': 'NEAR Protocol',
   'symbol': 'NEAR',
   'category': 'coin',
   'description': 'NEAR Protocol (NEAR) is a cryptocurrency launched in 2020. NEAR Protocol has a current supply of 1,000,000,000 with 617,213,739 in circulation. The last known price of NEAR Protocol is 17.38616033 USD and is up 2.04 over the last 24 hours. It is currently trading on 88 active market(s) with $921,895,821.20 traded over the last 24 hours. More information can be found at https://near.org/.',
   'slug': 'near-protocol',
   'logo': 'https://s2.coinmarketcap.com/static/img/coins/64x64/6535.png',
   'subreddit': '',
   'notice': 'Learn more about "[Near Protocol](https://coinmarketcap.com/earn/project/near-protocol)" & Earn $NEAR',
   'tags': ['platform',
    'staking',
    'binance-smart-chain',
    'coinbase-ventures-p

In [26]:
a = { 'a' : [1,2,3,4], 'b' : [3,4,2,4,5,6,6]}
print(len(a))
s = set()
for _a in a :
    s.update( a[_a])

print( a[max( a, key= lambda x : len(set(a[x])) )]  )
print(s)

2
[3, 4, 2, 4, 5, 6, 6]
{1, 2, 3, 4, 5, 6}
